In [ ]:
with open('data.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    data_object2 = pickle.load(f)
data_object2

In [ ]:
#interactions
from itertools import combinations

crossvalidation = KFold(n_splits=10, shuffle=True, random_state=0)
regression = LinearRegression()

interactions = []

feat_combinations = combinations(X.columns, 2)

data = X.copy()
for i, (a, b) in enumerate(feat_combinations):
    data['interaction'] = data[a] * data[b]
    score = np.mean(cross_val_score(regression, data, y, scoring='r2', cv=crossvalidation))
    if score > baseline:
        interactions.append((a, b, round(score,3)))
    
    if i % 50 == 0:
        print(i)
            
print('Top 3 interactions: %s' %sorted(interactions, key=lambda inter: inter[2], reverse=True)[:3])

In [ ]:
# 3000
# Top 3 interactions: [('sqft_living', 'zipcode_98023', 0.808), ('sqft_living', 'zipcode_98042', 0.808), ('sqft_living', 'zipcode_98103', 0.808)]

In [ ]:
regression = LinearRegression()
crossvalidation = KFold(n_splits=10, shuffle=True, random_state=1)
final = X.copy()

df_ohe = pd.get_dummies(cropped_df[cat_cols], columns=cat_cols, drop_first=True)
df_ohe.columns = df_ohe.columns.str.replace('.', '_')

final['sqft_living*zipcode_98023'] = final['sqft_living'] * final['zipcode_98023']
final['sqft_living*zipcode_98042'] = final['sqft_living'] * final['zipcode_98042']
final['sqft_living*zipcode_98103'] = final['sqft_living'] * final['zipcode_98103']

final_model_score = np.mean(cross_val_score(regression, final, y, scoring='r2', cv=crossvalidation))
final_model_score

In [ ]:
fig, ax = plt.subplots(figsize = (16,10))
ax.scatter(sqft_living_outliers_df.long, sqft_living_outliers_df.lat, zorder=1, alpha= 1, c=sqft_living_outliers_df.price, s=10)
ax.set_xlim(BBox[0],BBox[1])
ax.set_ylim(BBox[2],BBox[3])
ax.set_title('Plotting Data on Map')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.plot(-122.2,47.5983,c='r',markersize=5, marker='o') # Most expensive house
ax.imshow(map_img, zorder=0, extent = BBox, aspect= 'equal');

In [ ]:
#import chart_studio.plotly as py
import plotly.offline as py
import plotly.graph_objects as go

mapbox_access_token = 'pk.eyJ1IjoidmFkaW1icyIsImEiOiJja240c3JqaTExaXh4Mm5vMmx3NDNwOHUwIn0.tHSRuXCZ3UQxUnwNQRyZCQ'

#df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/Nuclear%20Waste%20Sites%20on%20American%20Campuses.csv')
site_lat = cropped_df.long
site_lon = cropped_df.lat
locations_name = cropped_df.price

data = [
    go.Scattermapbox(
        lat=list(site_lat),
        lon=list(site_lon),
        mode='markers',
        marker=dict(
            size=17,
            color='rgb(255, 0, 0)',
            opacity=0.7
        ),
        text=locations_name,
        hoverinfo='text'
    ),
    go.Scattermapbox(
        lat=site_lat,
        lon=site_lon,
        mode='markers',
        marker=dict(
            size=8,
            color='rgb(242, 177, 172)',
            opacity=0.7
        ),
        hoverinfo='none'
    )
]


layout = go.Layout(
    title='Nuclear Waste Sites on Campus',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=47.6,
            lon=-122
        ),
        pitch=0,
        zoom=9,
        style='light'
    ),
)

fig = dict(data=data, layout=layout)

#py.iplot(fig, filename='jupyter-Nuclear Waste Sites on American Campuses')
py(fig)
#fig.layout.update()
#fig.show()
#plot( fig, filename = 'candlestick-test-3.html', validate = False )